In [ ]:
import pandas as pd
from os import walk

In [ ]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
datasets_names

In [ ]:
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print('adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print('adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
print('total rows:', len(dataFrame))
dataFrame.head()

In [ ]:
import numpy as np

## Limpieza

In [ ]:
# drop outliers
from scipy import stats
dataFrame.loc[:,  'price'] = dataFrame.price.apply(lambda x: x[1:-3].replace(',', '')).astype(int)
dataFrame = dataFrame[dataFrame.price!=0]
dataFrame = dataFrame[(dataFrame.number_of_reviews>4)]
dataFrame = dataFrame[(np.abs(stats.zscore(dataFrame[['price']])) < 3)]

In [ ]:
len(dataFrame)

In [ ]:
import importlib
import load_transform_pipeline #import the module here, so that it can be reloaded.
importlib.reload(load_transform_pipeline)

transform_pipeline = load_transform_pipeline.load_model('../models/transformDataPipeline__not_fitted.pkl')

transform_pipeline

In [ ]:
from sklearn.model_selection import train_test_split

y = dataFrame.price
X = dataFrame
X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
transform_pipeline.fit(X_train_df)

In [ ]:
X_train = transform_pipeline.transform(X_train_df)

In [ ]:
X_train.shape

In [ ]:
X_test = transform_pipeline.transform(X_test_df)

In [ ]:
X_test.shape

In [ ]:
data = {
    'X_train_df': X_train_df,
    'X_test_df': X_test_df,
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'transform_pipeline': transform_pipeline
}
import pickle
with open("../models/data.pkl", 'wb') as pfile:
    pickle.dump(data, pfile, protocol=pickle.HIGHEST_PROTOCOL)